In [61]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import geopandas as gpd
import os
import re
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor

In [62]:
import pyarrow.parquet as pq

In [63]:
green_tripdata_2023 = r'C:\Users\yin li\OneDrive\Desktop\taxi_project\green_tripdata_2023'

df231 = pd.read_parquet(green_tripdata_2023 + '/' +'green_tripdata_2023-01.parquet')

print(df231.columns)

Index(['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime',
       'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID',
       'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge',
       'total_amount', 'payment_type', 'trip_type', 'congestion_surcharge'],
      dtype='object')


In [64]:
df231.dtypes

VendorID                          int64
lpep_pickup_datetime     datetime64[us]
lpep_dropoff_datetime    datetime64[us]
store_and_fwd_flag               object
RatecodeID                      float64
PULocationID                      int64
DOLocationID                      int64
passenger_count                 float64
trip_distance                   float64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
ehail_fee                        object
improvement_surcharge           float64
total_amount                    float64
payment_type                    float64
trip_type                       float64
congestion_surcharge            float64
dtype: object

In [65]:
df231.describe()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
count,68211.000000,68211,68211,63887.000000,68211.000000,68211.000000,63887.000000,68211.000000,68211.000000,68211.000000,68211.000000,68211.000000,68211.000000,68211.000000,68211.000000,63887.000000,63877.000000,63887.000000
mean,1.863028,2023-01-16 20:10:55.679523,2023-01-16 20:29:01.515767,1.117160,98.549735,138.429901,1.315870,8.114852,16.603545,0.825431,0.588340,2.139012,0.169627,0.951625,21.789377,1.373613,1.021933,0.726748
min,1.000000,2009-01-01 20:21:27,2009-01-02 11:07:31,1.000000,1.000000,1.000000,0.000000,0.000000,-70.000000,-2.500000,-0.500000,-10.500000,0.000000,-1.000000,-71.500000,1.000000,1.000000,-2.750000
25%,2.000000,2023-01-09 11:59:47.500000,2023-01-09 12:16:37.500000,1.000000,74.000000,74.000000,1.000000,1.110000,9.300000,0.000000,0.500000,0.000000,0.000000,1.000000,12.900000,1.000000,1.000000,0.000000
50%,2.000000,2023-01-17 08:40:42,2023-01-17 08:56:38,1.000000,75.000000,138.000000,1.000000,1.850000,13.500000,0.000000,0.500000,1.600000,0.000000,1.000000,17.900000,1.000000,1.000000,0.000000
75%,2.000000,2023-01-24 15:52:30,2023-01-24 16:06:56,1.000000,129.000000,219.000000,1.000000,3.210000,19.800000,1.000000,0.500000,3.330000,0.000000,1.000000,26.150000,2.000000,1.000000,2.750000
max,2.000000,2023-02-01 03:10:05,2023-02-01 17:27:05,99.000000,265.000000,265.000000,9.000000,120098.840000,490.000000,12.500000,2.750000,222.220000,36.050000,1.000000,491.000000,5.000000,2.000000,2.750000
std,0.343820,NaN,NaN,1.372913,61.244314,76.761311,0.979054,585.105955,13.470121,1.269904,0.385819,3.052710,1.099789,0.196875,15.457115,0.508039,0.146465,1.212836


Issues of the dataframe that need to clean
1. datetime needs to be DataFrame datetime type
2. pick up datetime should be within the certain month
3. RatecodeID should be in the valid values
4. trip_distance has the biggest std, which means the very skewed data. for trip_distance greater than 100 miles, i will use (fare_amount - 2.5) / 2.5 to replace the value of trip_distance.
5. all the values of ehail_fees are none, dropping the column



In [66]:
# Folder containing the Parquet files
folder_path = r'C:\Users\yin li\OneDrive\Desktop\taxi_project\green_tripdata_2023'
output_folder = r'C:\Users\yin li\OneDrive\Desktop\taxi_project\cleaned_data'

# Define the set of RatecodeID values to keep
ratecodeid_set = {1.0, 2.0, 3.0, 4.0, 5.0}

# Function to calculate new trip_distance
def calculate_trip_distance(fare_amount):
    return (fare_amount - 2.5) / 2.5

# Function to extract the month from the filename
def extract_month(filename):
    match = re.search(r'green_tripdata_2023-(\d{2})', filename)
    if match:
        return int(match.group(1))
    else:
        raise ValueError(f"Month not found in filename: {filename}")

# Function to clean the data
def clean_data(df, month):
    # Convert datetime columns
    df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])
    df['lpep_dropoff_datetime'] = pd.to_datetime(df['lpep_dropoff_datetime'])
    
    # Calculate trip duration in minutes
    df['duration_minutes'] = (df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']).dt.total_seconds() / 60
    
    # Define the start and end of the month
    start_date = f'2023-{month:02d}-01'
    end_date = f'2023-{month + 1:02d}-01' if month < 12 else '2024-01-01'
    
    # Filter by pickup datetime within the month
    df = df[(df['lpep_pickup_datetime'] >= start_date) & (df['lpep_pickup_datetime'] < end_date)]
    
    # Drop the ehail_fee column
    df = df.drop(columns=['ehail_fee'])
    
    # Filter by RatecodeID
    df = df[df['RatecodeID'].isin(ratecodeid_set)]
    
    # Identify and correct trip_distance over 100
    mask = df['trip_distance'] > 100
    df.loc[mask, 'trip_distance'] = df.loc[mask, 'fare_amount'].apply(calculate_trip_distance)
    
    return df

# List all Parquet files in the folder
all_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.parquet')]

# Print all file paths
print("All files:", all_files)

# Read and clean each Parquet file
dfs = []
for file in all_files:
    month = extract_month(file)
    df = pd.read_parquet(file)
    cleaned_df = clean_data(df, month)
    dfs.append(cleaned_df)

# Combine all cleaned DataFrames
combined_df = pd.concat(dfs, ignore_index=True)

# Print the combined DataFrame
print("Combined DataFrame:")
print(combined_df)

# Calculate skewness for numeric columns
numeric_columns = combined_df.select_dtypes(include=['number']).columns
skewness = combined_df[numeric_columns].skew()



# Function to save DataFrame to Parquet file with structured path
def save_to_parquet(df, year, month, output_folder):
    month_str = f'{month:02d}'
    path = os.path.join(output_folder, f'year={year}', f'month={month_str}')
    os.makedirs(path, exist_ok=True)
    file_path = os.path.join(path, f'data_{year}_{month_str}.parquet')
    df.to_parquet(file_path)
    print(f"Saved to {file_path}")

# Split combined DataFrame by month and save each to the appropriate folder structure
for month in range(1, 13):
    monthly_df = combined_df[(combined_df['lpep_pickup_datetime'].dt.month == month)]
    if not monthly_df.empty:
        save_to_parquet(monthly_df, 2023, month, output_folder)


All files: ['C:\\Users\\yin li\\OneDrive\\Desktop\\taxi_project\\green_tripdata_2023\\green_tripdata_2023-01.parquet', 'C:\\Users\\yin li\\OneDrive\\Desktop\\taxi_project\\green_tripdata_2023\\green_tripdata_2023-02.parquet', 'C:\\Users\\yin li\\OneDrive\\Desktop\\taxi_project\\green_tripdata_2023\\green_tripdata_2023-03.parquet', 'C:\\Users\\yin li\\OneDrive\\Desktop\\taxi_project\\green_tripdata_2023\\green_tripdata_2023-04.parquet', 'C:\\Users\\yin li\\OneDrive\\Desktop\\taxi_project\\green_tripdata_2023\\green_tripdata_2023-05.parquet', 'C:\\Users\\yin li\\OneDrive\\Desktop\\taxi_project\\green_tripdata_2023\\green_tripdata_2023-06.parquet', 'C:\\Users\\yin li\\OneDrive\\Desktop\\taxi_project\\green_tripdata_2023\\green_tripdata_2023-07.parquet', 'C:\\Users\\yin li\\OneDrive\\Desktop\\taxi_project\\green_tripdata_2023\\green_tripdata_2023-08.parquet', 'C:\\Users\\yin li\\OneDrive\\Desktop\\taxi_project\\green_tripdata_2023\\green_tripdata_2023-09.parquet', 'C:\\Users\\yin li\\OneDr

In [67]:
print("Combined DataFrame:")
print(combined_df)

Combined DataFrame:
        VendorID lpep_pickup_datetime lpep_dropoff_datetime  \
0              2  2023-01-01 00:26:10   2023-01-01 00:37:11   
1              2  2023-01-01 00:51:03   2023-01-01 00:57:49   
2              2  2023-01-01 00:35:12   2023-01-01 00:41:32   
3              1  2023-01-01 00:13:14   2023-01-01 00:19:03   
4              1  2023-01-01 00:33:04   2023-01-01 00:39:02   
...          ...                  ...                   ...   
731248         2  2023-12-31 23:23:58   2023-12-31 23:39:18   
731249         2  2023-12-31 23:05:16   2023-12-31 23:33:32   
731250         2  2023-12-31 23:33:08   2023-12-31 23:47:07   
731251         2  2023-12-31 23:51:30   2023-12-31 23:53:56   
731252         2  2023-12-31 23:54:37   2023-12-31 23:56:14   

       store_and_fwd_flag  RatecodeID  PULocationID  DOLocationID  \
0                       N         1.0           166           143   
1                       N         1.0            24            43   
2               

In [68]:
# Display the skewness
print("Skewness:")
print(skewness)

Skewness:
VendorID                 -2.089663
RatecodeID                4.696985
PULocationID              1.317572
DOLocationID              0.105430
passenger_count           3.597342
trip_distance             6.443385
fare_amount              33.391614
extra                     1.754568
mta_tax                   1.641177
tip_amount               12.421742
tolls_amount              7.512707
improvement_surcharge    -8.064449
total_amount             25.769767
payment_type              1.046312
trip_type                 5.039037
congestion_surcharge      1.033357
duration_minutes         16.216663
dtype: float64
